In [1]:
import numpy as np
import os
import json
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Final Model (YOLOv5 from Experiment 3)

In [19]:
%cd /content/drive/My Drive/ECE1508\ DL\ Project

/content/drive/My Drive/ECE1508 DL Project


In [20]:
%cd /content/drive/My Drive/ECE1508\ DL\ Project
# !pip install ultralytics
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

/content/drive/My Drive/ECE1508 DL Project


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-4-6 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7045186 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


## Heuristics

In [21]:
warm_to_hot_clothing = ['skirt', 'shorts', 'short sleeve top', 'sling dress','vest',
                        'vest dress', 'short sleeve dress', 'sling']
warm_clothing = ['short sleeve outwear'] # everything in warm_to_hot plus this
cool_clothing = ['trousers', 'long sleeve dress', 'long sleeve top', 'long sleeve outwear']

In [45]:
type('hi') == str

True

In [46]:
def heuristics_w_temp_range(detected_categories, temperature):
    # warm to hot
    if type(temperature) == str:
      temperature = int(temperature)
    if temperature <= 30 and temperature >= 25:
        print("Weather is warm to hot:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in warm_to_hot_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in warm_to_hot_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    elif temperature < 25 and temperature >= 20:
        print("Weather is warm:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in warm_to_hot_clothing or c in warm_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in warm_to_hot_clothing or c in warm_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    elif temperature < 20 and temperature >= 15:
        print("Weather is cool:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in cool_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in cool_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    else:
        print("Invalid temperature range! Enter temperature between 15 to 30 degrees celsius")


## Integration of Model and Heuristics

In [47]:
def is_outfit_appropriate(current_weather, image):
  prediction = model(image)
  prediction_str = str(prediction)
  start_index = prediction_str.index('352x352') + len('352x352')
  end_index = prediction_str.index('Speed')
  extracted_string = prediction_str[start_index:end_index].strip()

  predictions_list = extracted_string.split(',')
  # Process each prediction to extract the class name
  classes = [prediction.split(' ')[-2:] for prediction in predictions_list]

  # Join the class names into a list
  for i in range(len(classes)):
    if classes[i][0].isnumeric():
      classes[i][0] = ' '
  class_names = [' '.join(class_name).strip() for class_name in classes]

  heuristics_w_temp_range(class_names, current_weather)


In [24]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=

## Test System

In [50]:
image_ids = ['ex']
temps = [22, 10, 27]
test_images_dir = 'content/drive/My Drive/ECE1508\ DL\ Project'

for i in image_ids:
  for j in temps:
    # image = os.path.join(test_images_dir,  i + '.jpg')
    image = f'{i}.jpg'
    print(f'For {i}, and temperature {j} degrees celsius:')
    is_outfit_appropriate(j, image)
    print()


For ex, and temperature 22 degrees celsius:
Weather is warm:
skirt is suitable for this weather!
vest dress is suitable for this weather!

For ex, and temperature 10 degrees celsius:
Invalid temperature range! Enter temperature between 15 to 30 degrees celsius

For ex, and temperature 27 degrees celsius:
Weather is warm to hot:
skirt is suitable for this weather!
vest dress is suitable for this weather!



In [51]:
import gradio as gr
def run_demo():
  input_box = gr.Textbox(label="Enter Temperature outside", lines=1, placeholder="Temperature in degrees Celsius.")
  output_box = gr.Textbox(label="Outfit Weather Suitability Predictions:")

  demo = gr.Interface(
      fn=is_outfit_appropriate,
      inputs=[input_box, gr.Image(type="pil")],
      examples=[["20","ex.jpg"]],
      outputs=[output_box])


  demo.launch(debug=True)

In [52]:
run_demo()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://303639e06aa98b1762.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Weather is warm:
skirt is suitable for this weather!
vest dress is suitable for this weather!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://303639e06aa98b1762.gradio.live
